# Clustering Crypto

In [17]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import urllib.request, json

### Fetching Cryptocurrency Data

In [18]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [8]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [26]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
r = requests.get(url)

data = r.json()


In [27]:
list(data.keys())

['Response',
 'Message',
 'Data',
 'BaseImageUrl',
 'BaseLinkUrl',
 'RateLimit',
 'HasWarning',
 'Type']

In [28]:
data = data['Data']

In [29]:
crypto_df = pd.DataFrame.from_dict(data)

In [30]:
crypto_df = crypto_df.T
crypto_df.head()

,Algorithm,AssetLaunchDate,AssetTokenStatus,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,DecimalPoints,...,PlatformType,ProofType,Rating,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinsMined,Url
42,Scrypt,0000-00-00,N/A,0,0,0,NaN,42 Coin,1427211129,NaN,...,NaN,PoW/PoS,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",NaN,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",0,/coins/42/overview
300,N/A,2017-07-01,N/A,0,0,0,ETH,300 token,1517935016,18,...,token,N/A,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,/coins/300/overview
365,X11,0000-00-00,N/A,0,0,0,NaN,365Coin,1480032918,NaN,...,NaN,PoW/PoS,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",NaN,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",0,/coins/365/overview
404,Scrypt,0000-00-00,N/A,0,0,0,NaN,404Coin,1466100361,NaN,...,NaN,PoW/PoS,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",NaN,602,False,404,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",0,/coins/404/overview
433,N/A,NaN,Finished,NaN,NaN,NaN,NaN,433 Token,1541597321,NaN,...,NaN,N/A,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",NaN,3505,False,433,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",NaN,/coins/433/overview


### Data Preprocessing

In [38]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df
keep_col = ['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined']


In [39]:
crypto_df[keep_col]

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined
42,42 Coin,Scrypt,True,PoW/PoS,0
300,300 token,N/A,True,N/A,300
365,365Coin,X11,True,PoW/PoS,0
404,404Coin,Scrypt,True,PoW/PoS,0
611,SixEleven,SHA-256,True,PoW,0
808,808,SHA-256,True,PoW/PoS,0
888,Octocoin,N/A,True,PoW,0
1337,EliteCoin,X13,True,PoW/PoS,0
2015,2015 coin,X11,True,PoW/PoS,0
BTCD,BitcoinDark,SHA-256,True,PoW/PoS,NaN


In [41]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]


In [42]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [43]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns = 'IsTrading')

In [45]:
# Remove rows with at least 1 null value
crypto_df.isnull().sum()

Algorithm                  0
AssetLaunchDate         1208
AssetTokenStatus           0
BlockNumber             1208
BlockReward             1208
BlockTime               1208
BuiltOn                 1468
CoinName                   0
ContentCreatedOn           0
DecimalPoints           1460
Description                0
Difficulty              1434
FullName                   0
Id                         0
ImageUrl                   0
IsUsedInDefi            1208
IsUsedInNft             1208
MaxSupply               1208
MktCapPenalty           1208
Name                       0
NetHashesPerSecond      1210
PlatformType            1363
ProofType                  0
Rating                     0
SmartContractAddress    1472
SortOrder                  0
Sponsored                  0
Symbol                     0
Taxonomy                   0
TotalCoinsMined         1208
Url                        0
dtype: int64

In [46]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [48]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']

In [49]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = pd.DataFrame(
    data = crypto_df, columns = ['CoinName'])

In [50]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns = 'CoinName')

In [79]:
# Create dummy variables for text features
crypto_dummy = pd.get_dummies(crypto_df, columns=['ImageUrl', 'ProofType'])
crypto_dummy.head()

,Algorithm,AssetLaunchDate,AssetTokenStatus,BlockNumber,BlockReward,BlockTime,BuiltOn,ContentCreatedOn,DecimalPoints,Description,...,ProofType_Proof of Authority,ProofType_Proof of Stake,ProofType_Proof of Staked Authority (PoSA),ProofType_Proof of Trust,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW,ProofType_dPoW/PoW,ProofType_mPoW
BTCD,SHA-256,NaN,NaN,NaN,NaN,NaN,NaN,1427711372,NaN,Bitcoin Dark (BTCD) is a PoW and PoS hybrid al...,...,0,0,0,0,0,0,0,0,0,0
CRAIG,X11,NaN,NaN,NaN,NaN,NaN,NaN,1427711632,NaN,CraigCoin (CRAIG) is a 100% pure PoS or Proof ...,...,0,0,0,0,0,0,0,0,0,0
XBS,X11,NaN,NaN,NaN,NaN,NaN,NaN,1428671859,NaN,To create the infrastructure necessary to impl...,...,0,0,0,0,0,0,0,0,0,0
XPY,SHA-256,NaN,NaN,NaN,NaN,NaN,NaN,1428671950,NaN,Paycoin is an SHA256 cryptocurrency that switc...,...,0,0,0,0,0,0,0,0,0,0
PRC,Scrypt,NaN,NaN,NaN,NaN,NaN,NaN,1424886122,NaN,ProsperCoin is a PoW cryptocurrency with a mod...,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# Standardize data
from sklearn.preprocessing import StandardScaler
import numpy as np
crypto_scaled = StandardScaler().fit_transform(crypto_dummy)
print (crypto_scaled)

ValueError: could not convert string to float: 'SHA-256'

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components


In [ ]:
# Create a DataFrame with the principal components data


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values


# Create the Elbow Curve using hvPlot


Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
